In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score

In [2]:
import torch

In [3]:
a = torch.tensor([1,2,3,4])
b = torch.tensor([5,6,7,8])
torch.stack([a,b],1)

tensor([[1, 5],
        [2, 6],
        [3, 7],
        [4, 8]])

In [4]:
train_data = pd.read_csv("../stance_code_local/consistency_data.csv")
test_data = pd.read_csv('../stance_code_local/inconsistency_data.csv')

In [5]:
train_data

,undermine_score,support_score,cp_distance,cop_distance,predict_label,distance_label,gold
0,4.980137,-5.918371,1.923178,0.001339,0,0,0
1,-4.051531,3.098645,0.004558,0.010478,1,1,1
2,-1.103358,5.317815,0.293655,1.842567,1,1,1
3,-2.957279,5.612340,0.010343,1.754936,1,1,1
4,-2.915035,5.855358,0.004241,1.868177,1,1,1
...,...,...,...,...,...,...,...
2184,2.155837,-1.537567,1.053408,0.922337,0,0,0
2185,6.712749,-5.522651,1.332932,0.112898,0,0,0
2186,-2.778294,2.587539,0.518143,1.368011,1,1,1
2187,-3.157147,3.050995,0.483791,1.372143,1,1,1


In [6]:
test_data

,undermine_score,support_score,cp_distance,cop_distance,predict_label,distance_label,gold,Label,claim,perspective,opposite_perspective
0,-4.223587,3.430138,0.020014,0.012879,1,0,1,1,School Day Should Be Extended,Lessons would feel less broken.,Lessons would not feel less broken.
1,-1.659985,2.563246,0.277413,0.157976,1,0,1,1,School Day Should Be Extended,Less need for homework.,More need for homework.
2,6.975863,-2.372392,1.920776,1.928768,0,1,1,1,Chess must be at the Olympics,"Chess is currently not an Olympic sport, but i...","Chess is currently an Olympic sport, but it sh..."
3,1.812300,-0.927047,1.096693,1.661179,0,1,1,1,Federal States are better than unitary nations,Citizens are better protected in federal states.,Citizens are not better protected in federal s...
4,2.363664,-1.478531,1.179334,1.422178,0,1,1,1,Federal States are better than unitary nations,Federal states are economically stronger,Federal states are not economically stronger
...,...,...,...,...,...,...,...,...,...,...,...
333,-3.776413,2.909049,0.075312,0.017392,1,0,0,0,NATO has succeeded in Afghanistan,the taliban is waiting and likley to regain co...,the taliban is not waiting and likley to regai...
334,0.627612,-0.557652,0.694943,1.455035,0,1,0,0,Illegal downloaders should be cut off from the...,"File-sharing will still happen, despite these ...","File-sharing will not still happen, despite th..."
335,2.762536,-1.856774,0.735328,0.786035,0,1,0,0,Illegal downloaders should be cut off from the...,Will file-sharing stop if downloaders are bann...,will not file-sharing stop if downloaders are...
336,0.033896,-0.449516,0.755636,1.142089,0,1,1,1,Illegal downloaders should be cut off from the...,Internet access is an enabler of rights not a ...,Internet access is an enabler of rights a righ...


In [7]:
train_x = train_data.drop(columns=['predict_label',"distance_label",'gold'],axis=1)
train_y = train_data['gold']

In [8]:
test_x = test_data.drop(columns=['gold','predict_label',"distance_label","Label", "claim","perspective","opposite_perspective"],axis=1)
test_y = test_data['gold']

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(train_x)
train_x = sc.transform(train_x)
test_x = sc.transform(test_x)

In [10]:
from sklearn.model_selection import cross_val_score

In [11]:
model = LogisticRegression(solver='liblinear')
## lbfgs,liblinear,newton-cg,sag,saga

In [12]:
from sklearn.model_selection import GridSearchCV
penaltys = ['l1','l2']
# Cs = [0.001, 0.01, 0.1,0.5, 1, 5, 10, 50, 100, 500, 1000, 5000, 10000]

Cs = [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000, 5000, 10000]
print(Cs)
tuned_parameters = dict(penalty = penaltys, C = Cs)
 
grid= GridSearchCV(model, tuned_parameters,cv=5, scoring='neg_log_loss')
grid.fit(train_x,train_y)
 
grid.cv_results_
 
print(-grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

[0.001, 0.01, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000, 5000, 10000]
0.44396327480618497
{'C': 0.1, 'penalty': 'l2'}
LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)


In [13]:
model = grid.best_estimator_
model.fit(train_x,train_y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
print('Coefficient of model :', model.coef_)

Coefficient of model : [[-0.76649112  0.60163997 -0.29782019  0.20366811]]


In [15]:
print('Intercept of model',model.intercept_)

Intercept of model [0.06911548]


In [16]:
predict_train = model.predict(train_x)

In [17]:
# accuracy_train = accuracy_score(train_y,predict_train)
# print('accuracy_score on train dataset : ', accuracy_train)

In [18]:
# predict_test = model.predict(test_x)
# accuracy_test = accuracy_score(test_y,predict_test)
# print('accuracy_score on test dataset : ', accuracy_test)

In [20]:
predict_test = model.predict(test_x)
f1 = f1_score(test_y,predict_test)
print(f1)

0.6140350877192982


In [21]:
accuracy_score(test_y,predict_test)

0.6094674556213018